In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [3]:
import google

<module 'google' (namespace)>

In [33]:
import pandas as pd
import numpy as np

# Reading Data

In [34]:
def get_data_from_worksheet(worksheet):
    rows = worksheet.get_all_values()
    
    start_marker = "#"
    for start_row_index, r in enumerate(rows):
        if r[0] == start_marker:
            break

    data = rows[start_row_index:]

    # Convert to a DataFrame and render.
    df = pd.DataFrame(data[1:], columns=[str(s).lower() for s in data[0]])
    df = df[df.exercise != '']
    return df

In [35]:
def load_data_from_spreadsheet(spreadsheet):
    dfs = []
    for worksheet in spreadsheet.worksheets():
        df = get_data_from_worksheet(worksheet)
        df.insert(0, column="weekday", value=worksheet.title)
        dfs.append(df)

    return pd.concat(dfs, axis=0)

In [36]:
spreadsheet = gc.open("Week1")
load_data_from_spreadsheet(spreadsheet)

,weekday,#,exercise,sets,reps,load,tempo,rpe,note,
0,Monday,A,Goblet squat DB,2,16,16,,8,Goblet dřepy jsou s úchopem činky před tělem. ...,NaN
1,Monday,B,Split squat - excentric,2,12,1,401,8,Split squat už jsme taky spolu dělali na posil...,NaN
2,Monday,C,Bridges with dumbell,2,20,16,,6,Mosty s nohama opřenýma o podlahu. Činku si de...,NaN
3,Monday,D,Single leg bridges,2,12,1,,9,"Stejné jako předchozí, ale bez činky a na jedn...",NaN
4,Monday,E,Push ups - bear position,2,12,1,,9,Kliky v pozici medvěda. Je to pozice s opřeným...,NaN
5,Monday,F,TRX row,2,16,1,,6,"Přítahy na TRX, náklon si zvol podle sebe podl...",NaN
0,Tuesday,A,Dead bug + arm/leg mvmnt,2,20,1,401,8,"Mrtvé brouky máš i v rozvičce, tak tam je může...",Ako sa to počíta? Keď natiahnem raz pravú a ra...
1,Tuesday,B,Pullover,2,15,10,401,6,"Udělej si pevný pullover. To znamená, že nohy ...",
2,Tuesday,C,Side plank,2,30,1,,4,Bude to výdrž na boku v podporu na předloktí. ...,
3,Tuesday,D,Side lunges,2,12,1,,7,"Výpady do strany, při kroku do strany si troch...",


In [37]:
def load_data(names):
    dfs = []
    for name in names:
        spreadsheet = gc.open(name)
        df = load_data_from_spreadsheet(spreadsheet)
        df.insert(0, column="week", value=name)
        dfs.append(df)

    return pd.concat(dfs, axis=0)

In [38]:
df = load_data(["Week1", "Week2"])

In [39]:
set(df.exercise)

{'Back lunges',
 'Bridge - Izo',
 'Bridges with dumbell',
 'Calf raises',
 'DB/KB deadlift',
 'Dead bug + arm/leg mvmnt',
 'Dumbell deadlift',
 'Front lunge - bottom izo hold',
 'Goblet squat',
 'Goblet squat DB',
 'Pullover',
 'Push ups - bear position',
 'Push ups - excentric',
 'Quadraped walk fw/bw',
 'Quadraped walk side',
 'Shoulder push ups',
 'Side lunges',
 'Side plank',
 'Side plank - hip abductions',
 'Single - calf raises',
 'Single leg bridges',
 'Single leg deadlift',
 'Sit ups - excentrics',
 'Split squat - excentric',
 'Squat - excentric',
 'Squat bottom izo hold',
 'TRX delts row',
 'TRX row',
 'TRX single arm row'}

In [40]:
df[df.exercise == "Pullover"]

,week,weekday,#,exercise,sets,reps,load,tempo,rpe,note,
1,Week1,Tuesday,B,Pullover,2,15,10,401,6,"Udělej si pevný pullover. To znamená, že nohy ...",
2,Week1,Thursday,C,Pullover,2,15,10,401,5,"Udělej si pevný pullover. To znamená, že nohy ...",
1,Week2,Tuesday,B,Pullover,3,12,15,401,,Opět pevně zapřít hlavně horní hranu lopatky a...,NaN
2,Week2,Thursday,C,Pullover,3,15,15,401,,,NaN


In [41]:
for col in ["sets", "reps", "load", "rpe"]:
    df[col] = pd.to_numeric(df[col])

df.dtypes

week         object
weekday      object
#            object
exercise     object
sets          int64
reps          int64
load          int64
tempo        object
rpe         float64
note         object
             object
dtype: object

In [42]:
df1 = df.loc[:, : "rpe"]
df1["volume"] = df1["sets"] * df1["reps"] * df1["load"]
df1.head()

,week,weekday,#,exercise,sets,reps,load,tempo,rpe,volume
0,Week1,Monday,A,Goblet squat DB,2,16,16,,8.0,512
1,Week1,Monday,B,Split squat - excentric,2,12,1,401,8.0,24
2,Week1,Monday,C,Bridges with dumbell,2,20,16,,6.0,640
3,Week1,Monday,D,Single leg bridges,2,12,1,,9.0,24
4,Week1,Monday,E,Push ups - bear position,2,12,1,,9.0,24


In [43]:
gb = df1.groupby(["week", "exercise"])["volume"]
gb.agg("sum").unstack("exercise")

exercise,Back lunges,Bridge - Izo,Bridges with dumbell,Calf raises,DB/KB deadlift,Dead bug + arm/leg mvmnt,Dumbell deadlift,Front lunge - bottom izo hold,Goblet squat,Goblet squat DB,Pullover,Push ups - bear position,Push ups - excentric,Quadraped walk fw/bw,Quadraped walk side,Shoulder push ups,Side lunges,Side plank,Side plank - hip abductions,Single - calf raises,Single leg bridges,Single leg deadlift,Sit ups - excentrics,Split squat - excentric,Squat - excentric,Squat bottom izo hold,TRX delts row,TRX row,TRX single arm row
week,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Week1,40.0,512.0,640.0,105.0,NaN,40.0,512.0,24.0,NaN,512.0,600.0,24.0,16.0,90.0,135.0,16.0,48.0,60.0,40.0,NaN,24.0,56.0,30.0,24.0,320.0,320.0,24.0,32.0,20.0
Week2,150.0,640.0,960.0,75.0,640.0,20.0,NaN,120.0,640.0,NaN,1215.0,39.0,16.0,90.0,135.0,16.0,192.0,90.0,60.0,40.0,30.0,288.0,16.0,30.0,384.0,384.0,24.0,48.0,24.0


In [45]:
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

In [47]:
gb = df1.groupby(["week", "weekday"])["volume"]
gb.agg("sum").unstack("weekday")[weekdays]

weekday,Monday,Tuesday,Wednesday,Thursday,Friday
week,,,,,
Week1,1256,559,920,589,940
Week2,1747,847,1216,1086,1470


# Writing Data

In [12]:
# out_spreadsheet = gc.create("summary")
# ws = out_spreadsheet.add_worksheet("everything", 10, 10)

In [13]:
out_spreadsheet = gc.open("summary")

In [14]:

ws = out_spreadsheet.get_worksheet(1)

In [22]:
df1 = pd.DataFrame({"a": [1, 2, 3], "b": [4., 5., np.NaN]})
df1

,a,b
0,1,4.0
1,2,5.0
2,3,NaN


In [28]:
ws.update(df.fillna("").values.tolist())

{'spreadsheetId': '1T5SDN-mwZ0yLEpm_GckukdOcDZAk2mMRfMqDoPjZ5ww',
 'updatedCells': 660,
 'updatedColumns': 11,
 'updatedRange': 'everything!A1:K60',
 'updatedRows': 60}